<a href="https://colab.research.google.com/github/JoshDumo/PortolioManager/blob/main/Portfolio_manager.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Portfolio management
In this project we investigate portfolio management using python.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Getting data


In [ ]:
ticker_df = pd.read_csv("nyse.csv")

In [ ]:
ticker_df.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$128.635,-1.4850,-1.141%,3.928376e+10,United States,1999.0,1050123,Capital Goods,Biotechnology: Laboratory Analytical Instruments
1,AA,Alcoa Corporation Common Stock,$21.31,-0.6200,-2.827%,3.963193e+09,NaN,2016.0,2559525,Basic Industries,Aluminum
2,AAIC,Arlington Asset Investment Corp Class A (new),$3.9205,-0.0695,-1.742%,1.310532e+08,United States,NaN,161093,Consumer Services,Real Estate Investment Trusts
3,AAIC^B,Arlington Asset Investment Corp 7.00%,$22.64,-0.0400,-0.176%,NaN,United States,NaN,458,NaN,NaN
4,AAIC^C,Arlington Asset Investment Corp 8.250% Seies C...,$22.50,-0.0850,-0.376%,NaN,United States,NaN,4499,NaN,NaN


In [ ]:
is_US = ticker_df['Country'] == 'United States'
is_highVol = ticker_df['Volume'] > 2000000
is_highMCap = ticker_df['Market Cap'] > 1e+8
ticker_UShighVol = ticker_df[is_US & is_highVol & is_highMCap]
ticker_UShighVol_list = ticker_UShighVol['Symbol'].values.tolist()

In [ ]:
len(ticker_UShighVol_list)

203

In [ ]:
!pip install yahoofinancials

  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-cp36-none-any.whl size=15192 sha256=a5005004bd043f60fee450717a6403427871d7b99c5700d22d8f4eb18aa68040
  Stored in directory: /root/.cache/pip/wheels/d9/7e/cf/4977a8572d5247242a4b13018d1d36923024ba84236e0d28bc
Successfully built yahoofinancials


In [ ]:
from yahoofinancials import YahooFinancials
tech_stocks = ['AAPL', 'MSFT', 'INTC']
highVol_stocks = ticker_UShighVol_list
yahoo_financials_tech = YahooFinancials(tech_stocks)
#tech_stock_price_data = yahoo_financials_tech.get_stock_price_data()
daily_tech_stock_prices = yahoo_financials_tech.get_historical_price_data('2017-09-15', '2018-09-15', 'daily')



In [ ]:
highVol_stocks = ticker_UShighVol_list
yahoo_financials_highVol = YahooFinancials(highVol_stocks)
daily_highVol_stock_prices = yahoo_financials_highVol.get_historical_price_data('2013-01-02', '2020-12-15', 'daily')

In [ ]:
daily_highVol_stock_prices.keys()

dict_keys(['ABBV', 'ABT', 'ACIC', 'ADT', 'AES', 'AFL', 'AHT', 'AIG', 'AIV', 'ALB', 'AM', 'ATUS', 'AXTA', 'BA', 'BAC', 'BFT', 'BK', 'BMY', 'BSX', 'C', 'CAG', 'CCIV', 'CCL', 'CCO', 'CDE', 'CF', 'CFG', 'CHS', 'CLF', 'CLNY', 'CNC', 'CNP', 'CNX', 'COG', 'COP', 'COTY', 'CPE', 'CRK', 'CRM', 'CVS', 'CVX', 'CXW', 'CYH', 'D', 'DAL', 'DDD', 'DHF', 'DIS', 'DUK', 'DVN', 'ED', 'ELAN', 'EPD', 'EQT', 'ET', 'EXPR', 'F', 'FCX', 'FE', 'FHN', 'FINV', 'FLR', 'FNB', 'FRX', 'GE', 'GEN           ', 'GEO', 'GIS', 'GLW', 'GM', 'GME', 'GNW', 'GPK', 'GPS', 'GSAH', 'GTT', 'HAL', 'HBI', 'HD', 'HL', 'HPE', 'HPQ', 'HTA', 'HUYA', 'IBM', 'IFF', 'IGT', 'INVH', 'IPG', 'IPOE', 'IPOF', 'IVR', 'JNJ', 'JNPR', 'JPM', 'JWN', 'K', 'KAR', 'KEY', 'KIM', 'KMI', 'KO', 'KODK', 'KR', 'KSS', 'LLY', 'LOW', 'LTHM', 'LUMN', 'LUV', 'LVS', 'M', 'MAC', 'MD', 'MDT', 'MET', 'MFA', 'MGM', 'MMP', 'MO', 'MOS', 'MPC', 'MRK', 'MRO', 'MS', 'MUR', 'NCLH', 'NEE', 'NEM', 'NET', 'NGL', 'NKE', 'NLY', 'NOVA', 'NRG', 'NRZ', 'NUE', 'NVTA', 'NYCB', 'O', 'OM

In [ ]:
big_d = []
for symkey in daily_highVol_stock_prices.keys():
  p_info = daily_highVol_stock_prices[symkey].get('prices')
  if p_info != None:
    ps = [(p.get('formatted_date'),p.get('adjclose')) for p in p_info]
    p_df = pd.DataFrame(ps, columns=['date',symkey])
    p_df = p_df.set_index("date")
    big_d.append(p_df)
  else:
    continue
#print(pd.concat(big_d, axis=1))
stock_df = pd.concat(big_d, axis=1)


,ABBV,ABT,ADT,AES,AFL,AHT,AIG,AIV,ALB,AM,ATUS,AXTA,BA,BAC,BFT,BK,BMY,BSX,C,CAG,CCIV,CCL,CCO,CDE,CF,CFG,CHS,CLF,CLNY,CNC,CNP,CNX,COG,COP,COTY,CPE,CRK,CRM,CVS,CVX,...,SCHW,SE,SKT,SM,SNAP,SO,SPG,SQ,STPK,SWN,SYF,T,TFC,TGT,TJX,TPR,TTI,TWLO,TWO,U,UAA,UNP,UPS,USB,UWMC,V,VLO,VRT,WAB,WES,WFC,WM,WMB,WMT,WPF,WTI,WU,WY,X,XOM
2013-01-02,25.103134,27.273100,NaN,8.516671,22.132555,42.462776,30.981562,2.096377,56.299110,NaN,NaN,NaN,64.583839,10.661530,NaN,22.571970,26.123360,5.89,36.358650,18.789106,NaN,30.327806,3.243526,25.200001,32.771385,NaN,13.715101,34.538010,NaN,10.3400,14.039549,25.213322,22.894434,45.437935,NaN,51.900002,72.394943,42.792500,41.193867,78.484093,...,13.744130,NaN,23.307243,52.528496,NaN,29.904551,107.481178,NaN,NaN,33.349998,NaN,22.110373,23.289074,46.033428,19.464573,42.529373,7.576635,NaN,7.294277,NaN,12.0825,54.029587,59.881084,26.519627,NaN,36.668194,23.989239,NaN,42.794788,20.028854,27.206877,27.649210,21.523186,57.221653,NaN,15.649914,10.446214,21.877724,24.052210,63.030422
2013-01-03,24.895849,28.311268,NaN,8.470212,21.571774,42.501530,30.726778,2.103270,55.931358,NaN,NaN,NaN,64.919044,10.599492,NaN,22.320055,26.546642,5.95,36.482048,18.745396,NaN,29.947704,3.285050,23.850000,32.616428,NaN,13.812857,33.968456,NaN,10.2275,14.082485,25.961348,23.201374,45.407242,NaN,50.500000,71.874794,42.177502,41.276772,78.149925,...,13.725905,NaN,23.280218,52.836189,NaN,29.897732,106.620277,NaN,NaN,33.470001,NaN,22.123005,23.172899,47.082130,20.100319,42.390518,7.528984,NaN,7.275316,NaN,12.4150,54.105293,59.935997,26.149620,NaN,36.696491,23.475374,NaN,43.453548,19.978266,26.981770,27.746891,21.358738,56.858017,NaN,15.760516,10.271605,21.877724,23.615570,62.916748
2013-01-04,24.581341,28.141081,NaN,8.671516,21.301617,42.617748,30.828688,2.102504,56.588036,NaN,NaN,NaN,65.103409,10.732428,NaN,22.916265,26.634491,5.92,37.398727,18.882771,NaN,29.971962,3.252753,24.230000,33.347000,NaN,14.045948,33.363300,NaN,10.4000,14.182666,26.367880,23.303680,45.913734,NaN,52.400002,71.591072,42.402500,41.450912,78.562294,...,14.090470,NaN,23.205906,52.711185,NaN,30.164120,106.667358,NaN,NaN,34.340000,NaN,22.255665,23.389759,47.410824,20.091301,42.822525,7.690999,NaN,7.408053,NaN,12.3375,55.051636,60.085091,26.720713,NaN,36.996216,23.874300,NaN,43.309296,20.035183,27.121485,27.738749,21.592756,57.072884,NaN,15.972497,10.643600,22.057846,23.875690,63.208046
2013-01-07,24.631376,28.370836,NaN,8.400533,21.219751,42.230320,30.514462,2.108632,56.868214,NaN,NaN,NaN,63.796143,10.714706,NaN,22.580372,26.674421,5.98,37.433971,18.839066,NaN,29.874910,3.414237,23.510000,33.222076,NaN,13.857968,33.692577,NaN,10.6200,14.025240,25.839390,23.117659,45.407242,NaN,51.200001,72.489517,42.244999,41.484070,78.029053,...,13.917300,NaN,23.341022,52.663113,NaN,29.638151,107.427383,NaN,NaN,33.509998,NaN,22.356745,23.265835,47.974319,19.956038,42.915096,7.700530,NaN,7.376448,NaN,12.1075,54.790863,59.912472,26.479403,NaN,37.260513,23.225210,NaN,42.905388,20.180639,26.989532,27.649210,21.497889,56.527443,NaN,15.926414,10.377888,22.125393,23.884979,62.476223
2013-01-08,24.095289,28.379343,NaN,8.400533,21.465351,42.617748,30.276667,2.100973,55.835052,NaN,NaN,NaN,62.120163,10.617216,NaN,22.269672,26.746300,6.12,37.425171,18.826574,NaN,29.923437,3.321961,23.150000,32.735027,NaN,13.760217,33.834969,NaN,10.3300,13.932212,25.424719,22.768873,44.793327,NaN,52.799999,72.016655,42.492500,41.334820,77.680679,...,13.871731,NaN,23.361288,52.759266,NaN,29.556187,107.037292,NaN,NaN,32.730000,NaN,21.979225,23.103195,47.457783,19.613361,44.226543,7.938789,NaN,7.483903,NaN,12.1600,54.753021,59.747677,26.519627,NaN,37.607410,23.441566,NaN,42.857304,20.047831,26.942961,27.706192,21.232244,56.684483,NaN,15.788164,10.377888,22.275496,23.234674,62.867039


In [ ]:
stock_df.to_csv('highVolStocks200.csv')

## PyPortfolioOpt

In [ ]:
!pip install PyPortfolioOpt

     |████████████████████████████████| 61kB 2.8MB/s 
     |████████████████████████████████| 1.0MB 7.2MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for cvxpy: filename=cvxpy-1.1.10-cp36-cp36m-linux_x86_64.whl size=2683447 sha256=cf05835b66af511b2350f0c653c417a5bdf7e704d47a5f8cbca6c69eb0796f3e
  Stored in directory: /root/.cache/pip/wheels/64/d6/c9/9295499aefa5af80018eaabffd09216170a1349b45551fb8d5
Successfully built cvxpy
  Found existing installation: cvxpy 1.0.31
    Uninstalling cvxpy-1.0.31:
      Successfully uninstalled cvxpy-1.0.31


In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns


In [ ]:
mu = expected_returns.mean_historical_return(stock_df) 
S = risk_models.sample_cov(stock_df)

/usr/local/lib/python3.6/dist-packages/pypfopt/risk_models.py:69: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  "The covariance matrix is non positive semidefinite. Amending eigenvalues."
/usr/local/lib/python3.6/dist-packages/pypfopt/risk_models.py:88: UserWarning: Could not fix matrix. Please try a different risk model.
  "Could not fix matrix. Please try a different risk model.", UserWarning


In [ ]:
ef = EfficientFrontier(mu,S)
weights = ef.max_sharpe()

cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('ABBV', 0.0), ('ABT', 0.0), ('ADT', 0.0), ('AES', 0.02339), ('AFL', 0.0), ('AHT', 0.0), ('AIG', 0.0), ('AIV', 0.0), ('ALB', 0.0), ('AM', 0.0), ('ATUS', 0.0), ('AXTA', 0.0), ('BA', 0.0), ('BAC', 0.0), ('BFT', 0.09946), ('BK', 0.0), ('BMY', 0.0), ('BSX', 0.0), ('C', 0.0), ('CAG', 0.0), ('CCIV', 0.0), ('CCL', 0.0), ('CCO', 0.0), ('CDE', 0.0), ('CF', 0.0), ('CFG', 0.0), ('CHS', 0.0), ('CLF', 0.0), ('CLNY', 0.0), ('CNC', 0.0), ('CNP', 0.0), ('CNX', 0.0), ('COG', 0.0), ('COP', 0.0), ('COTY', 0.0), ('CPE', 0.0), ('CRK', 0.0), ('CRM', 0.0), ('CVS', 0.0), ('CVX', 0.0), ('CXW', 0.0), ('CYH', 0.0), ('D', 0.0), ('DAL', 0.0), ('DDD', 0.0), ('DHF', 0.0), ('DIS', 0.0), ('DUK', 0.0), ('DVN', 0.0), ('ED', 0.0), ('ELAN', 0.0), ('EPD', 0.0), ('EQT', 0.0), ('ET', 0.0), ('EXPR', 0.0), ('F', 0.0), ('FCX', 0.0), ('FE', 0.0), ('FHN', 0.0), ('FINV', 0.0), ('FLR', 0.0), ('FNB', 0.0), ('GE', 0.0), ('GEO', 0.0), ('GIS', 0.0), ('GLW', 0.0), ('GM', 0.05012), ('GME', 0.0), ('GNW', 0.0), ('GPK', 0.0), (

(40.2431073604792, 0.4579806581926586, 87.82708754385551)

In [ ]:
pip install pulp

     |████████████████████████████████| 40.6MB 100kB/s 


In [ ]:
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices


In [ ]:
portfolio_val = 8000
latest_price = get_latest_prices(stock_df)
weights = cleaned_weights

da = DiscreteAllocation(weights,latest_price,portfolio_val)

allocation, leftover = da.lp_portfolio()

print("Discrete Allocation: ", allocation)
print("Remaining amount: $", leftover)

Discrete Allocation:  {'AES': 9, 'BFT': 54, 'EXPR': 1, 'GM': 10, 'JNPR': 32, 'KODK': 1, 'NEE': 20, 'NKE': 10, 'QS': 28, 'RKT': 18, 'STPK': 13, 'TWLO': 1, 'WPF': 45}
Remaining amount: $ 0.9207839965820312


In [ ]:
portfolio_com = []
for k in allocation.keys():
  company = ticker_df.loc[ticker_df['Symbol'] == k]
  portfolio_com.append(company['Name'])
portfolio_com

[55    The AES Corporation Common Stock
 Name: Name, dtype: object,
 369    Foley Trasimene Acquisition Corp. II Class A C...
 Name: Name, dtype: object,
 1040    Express Inc. Common Stock
 Name: Name, dtype: object,
 1242    General Motors Company Common Stock
 Name: Name, dtype: object,
 1531    Juniper Networks Inc. Common Stock
 Name: Name, dtype: object,
 1596    Eastman Kodak Company Common New
 Name: Name, dtype: object,
 1901    NextEra Energy Inc. Common Stock
 Name: Name, dtype: object,
 1942    Nike Inc. Common Stock
 Name: Name, dtype: object,
 2295    QuantumScape Corporation Class A Common Stock
 Name: Name, dtype: object,
 2358    Rocket Companies Inc. Class A Common Stock
 Name: Name, dtype: object,
 2602    Star Peak Energy Transition Corp. Class A Comm...
 Name: Name, dtype: object,
 2773    Twilio Inc. Class A Common Stock
 Name: Name, dtype: object,
 2975    Foley Trasimene Acquisition Corp. Class A Comm...
 Name: Name, dtype: object]

In [ ]:
portfolio_df = pd.DataFrame(columns=['Company ticker','Company name','Discrete_Val_'+str(portfolio_val)])

In [ ]:
portfolio_df['Company ticker'] = list(allocation.keys())
portfolio_df['Company name'] = portfolio_com
portfolio_df['Discrete_Val_'+str(portfolio_val)] = list(allocation.values())
portfolio_df

,Company ticker,Company name,Discrete_Val_8000
0,AES,55 The AES Corporation Common Stock Name: N...,9
1,BFT,369 Foley Trasimene Acquisition Corp. II Cl...,54
2,EXPR,"1040 Express Inc. Common Stock Name: Name, ...",1
3,GM,1242 General Motors Company Common Stock Na...,10
4,JNPR,1531 Juniper Networks Inc. Common Stock Nam...,32
5,KODK,1596 Eastman Kodak Company Common New Name:...,1
6,NEE,1901 NextEra Energy Inc. Common Stock Name:...,20
7,NKE,"1942 Nike Inc. Common Stock Name: Name, dty...",10
8,QS,2295 QuantumScape Corporation Class A Commo...,28
9,RKT,2358 Rocket Companies Inc. Class A Common S...,18
